In [1]:
from __future__ import print_function
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from tensorflow.examples.tutorials.mnist import input_data
from skimage import transform
import numpy as np
import torch.optim as optim
# import tensorflow_datasets as tfds

mnist = input_data.read_data_sets("MNIST_data", one_hot=False)

/home/saqib/.conda/envs/tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/saqib/.conda/envs/tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/saqib/.conda/envs/tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/saqib/.conda/envs/tf/lib/python3.6/site-packages/tensorflow/pyth

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [22]:
class CNN(nn.Module):
    def __init__(self, n_classes):
        super(CNN, self).__init__()
        # conv layers: (in_channel size, out_channels size, kernel_size, stride, padding)
        self.conv1 = nn.Conv2d(1, 32, 5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(32, 16, 5, stride=1, padding=2)
        self.conv3 = nn.Conv2d(16, 8, 5, stride=1, padding=2)

        # max pooling (kernel_size, stride)
        self.pool = nn.MaxPool2d(2, 2)

        # fully conected layers:
        self.layer1 = nn.Linear(4*4*8, 64)
        self.layer2 = nn.Linear(64, 64)
        self.layer3 = nn.Linear(64, n_classes)

    def forward(self, x, training=True):
        # the autoencoder has 3 con layers and 3 deconv layers (transposed conv). All layers but the last have ReLu
        # activation function
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        print("Conv1: ", x.size())
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        print("Conv2: ", x.size())
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        print("Conv3: ", x.size())
        x = x.view(-1, 4 * 4 * 8)
        print("view: ", x.size())
        x = F.relu(self.layer1(x))
        print("l1: ", x.size())
        x = F.dropout(x, 0.5, training=training)
        x = F.relu(self.layer2(x))
        print("l2: ", x.size())
        x = F.dropout(x, 0.5, training=training)
        x = self.layer3(x)
        print("l3: ", x.size())
        return x

    def predict(self, x):
        # a function to predict the labels of a batch of inputs
        x = F.softmax(self.forward(x, training=False))
        return x

    def accuracy(self, x, y):
        # a function to calculate the accuracy of label prediction for a batch of inputs
        #   x: a batch of inputs
        #   y: the true labels associated with x
        prediction = self.predict(x)
        maxs, indices = torch.max(prediction, 1)
        acc = 100 * torch.sum(torch.eq(indices.float(), y.float()).float())/y.size()[0]
        return acc.cpu().data[0]


In [23]:
model = CNN(10)
model.cuda()

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(16, 8, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (layer1): Linear(in_features=128, out_features=64, bias=True)
  (layer2): Linear(in_features=64, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=10, bias=True)
)

In [35]:
t = getattr(model, 'conv1')
t

Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))

In [38]:
isinstance(t, torch.nn.modules.conv.Conv2d)

True

In [26]:
state = torch.load('cnn.pth')
model.load_state_dict(state)

<All keys matched successfully>

In [27]:
from torchsummary import summary

s = summary(model, (1, 32, 32), verbose=0)
s = str(s)

type(s)

print(s)

TypeError: summary() got an unexpected keyword argument 'verbose'

In [28]:
modules = []
for name, module in model.named_modules():
    modules.append(name)
print(modules) 

['', 'conv1', 'conv2', 'conv3', 'pool', 'layer1', 'layer2', 'layer3']


In [29]:
getattr(model, modules[1])

Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))

In [30]:
batch_size = 500   # Number of samples in each batch
epoch_num = 20      # Number of epochs to train the network
lr = 0.0005        # Learning rate

def next_batch(train=True):
    # A function to read the next batch of MNIST images and labels
    # Args:
    #   train: a boolean array, if True it will return the next train batch, otherwise the next test batch
    # Returns:
    #   batch_img: a pytorch Variable of size [batch_size, 1, 32, 32].
    #   batch_label: a pytorch Variable of size [batch_size, ].

    if train:
        batch_img, batch_label = mnist.train.next_batch(batch_size)
    else:
        batch_img, batch_label = mnist.test.next_batch(batch_size)

    # reshape the sample to a batch of images in pytorch order (batch, channels, height, width)
    batch_img = batch_img.reshape((-1, 1, 28, 28))

    # resize the images to (32, 32)
    resized_imgs = np.zeros((batch_img.shape[0], 1, 32, 32))
    for i in range(batch_img.shape[0]):
        resized_imgs[i, 0, ...] = transform.resize(batch_img[i, 0,...], (32, 32))

    batch_label = torch.from_numpy(batch_label).long()  # convert the numpy array into torch tensor
    batch_label = Variable(batch_label).cuda()          # create a torch variable and transfer it into GPU

    resized_imgs = torch.from_numpy(resized_imgs).float()     # convert the numpy array into torch tensor
    resized_imgs = Variable(resized_imgs).cuda()              # create a torch variable and transfer it into GPU
    return resized_imgs, batch_label

In [31]:
features, labels = next_batch(train=False)

np.argmax(model.predict(features).cpu().detach().numpy(), axis=1).shape

Conv1:  torch.Size([500, 32, 16, 16])
Conv2:  torch.Size([500, 16, 8, 8])
Conv3:  torch.Size([500, 8, 4, 4])


/home/saqib/.local/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


(500,)

In [32]:
weights = model.conv1.weight
weights

Parameter containing:
tensor([[[[ 1.9111e-03, -7.5466e-02, -8.3581e-02, -6.7995e-02, -2.1533e-01],
          [ 1.0316e-01,  2.1668e-01, -1.2940e-01,  2.4202e-01,  2.8774e-02],
          [ 2.1586e-01, -3.9543e-02,  2.9735e-01,  2.2145e-01,  1.7842e-01],
          [ 1.8348e-01,  6.5618e-02,  3.5390e-01,  8.3087e-02,  1.2447e-01],
          [ 1.6329e-01,  1.1724e-01,  2.5607e-01,  7.4599e-02,  1.0450e-01]]],


        [[[ 8.3825e-02,  5.9209e-02,  9.6223e-02,  2.2185e-01, -7.3424e-02],
          [-1.4050e-01,  5.5635e-02,  2.8236e-01, -1.1224e-01,  1.8013e-01],
          [-3.0268e-02,  1.7202e-01,  1.6379e-02,  2.7759e-01, -7.8112e-02],
          [ 7.4248e-02,  2.9403e-01,  9.7572e-02,  5.8801e-02,  1.0877e-01],
          [ 2.5658e-01,  2.5329e-02,  2.1847e-01,  1.8750e-01, -7.9955e-02]]],


        [[[ 1.1861e-01, -8.9525e-03, -1.6314e-01, -2.7319e-01,  1.1180e-01],
          [ 1.4242e-01, -1.8169e-01,  1.3020e-01, -1.9418e-01,  7.4939e-02],
          [-7.9088e-02,  4.6304e-02,  5.7971e-

In [33]:
weights.cpu().detach().numpy()

array([[[[ 1.91110442e-03, -7.54655898e-02, -8.35809186e-02,
          -6.79949671e-02, -2.15328813e-01],
         [ 1.03163399e-01,  2.16677651e-01, -1.29399538e-01,
           2.42020264e-01,  2.87740696e-02],
         [ 2.15863347e-01, -3.95433940e-02,  2.97348976e-01,
           2.21451953e-01,  1.78415030e-01],
         [ 1.83482409e-01,  6.56178743e-02,  3.53902370e-01,
           8.30869824e-02,  1.24465428e-01],
         [ 1.63289905e-01,  1.17239356e-01,  2.56065428e-01,
           7.45992586e-02,  1.04498647e-01]]],


       [[[ 8.38249996e-02,  5.92089072e-02,  9.62225199e-02,
           2.21849158e-01, -7.34242722e-02],
         [-1.40500262e-01,  5.56350201e-02,  2.82355994e-01,
          -1.12242937e-01,  1.80127814e-01],
         [-3.02680545e-02,  1.72015592e-01,  1.63788330e-02,
           2.77594447e-01, -7.81115294e-02],
         [ 7.42478967e-02,  2.94030994e-01,  9.75717232e-02,
           5.88006936e-02,  1.08765855e-01],
         [ 2.56579161e-01,  2.53290981e-02

In [34]:
model.conv1.weight = weights
model.conv1.weight

Parameter containing:
tensor([[[[ 1.9111e-03, -7.5466e-02, -8.3581e-02, -6.7995e-02, -2.1533e-01],
          [ 1.0316e-01,  2.1668e-01, -1.2940e-01,  2.4202e-01,  2.8774e-02],
          [ 2.1586e-01, -3.9543e-02,  2.9735e-01,  2.2145e-01,  1.7842e-01],
          [ 1.8348e-01,  6.5618e-02,  3.5390e-01,  8.3087e-02,  1.2447e-01],
          [ 1.6329e-01,  1.1724e-01,  2.5607e-01,  7.4599e-02,  1.0450e-01]]],


        [[[ 8.3825e-02,  5.9209e-02,  9.6223e-02,  2.2185e-01, -7.3424e-02],
          [-1.4050e-01,  5.5635e-02,  2.8236e-01, -1.1224e-01,  1.8013e-01],
          [-3.0268e-02,  1.7202e-01,  1.6379e-02,  2.7759e-01, -7.8112e-02],
          [ 7.4248e-02,  2.9403e-01,  9.7572e-02,  5.8801e-02,  1.0877e-01],
          [ 2.5658e-01,  2.5329e-02,  2.1847e-01,  1.8750e-01, -7.9955e-02]]],


        [[[ 1.1861e-01, -8.9525e-03, -1.6314e-01, -2.7319e-01,  1.1180e-01],
          [ 1.4242e-01, -1.8169e-01,  1.3020e-01, -1.9418e-01,  7.4939e-02],
          [-7.9088e-02,  4.6304e-02,  5.7971e-

In [ ]:
features.shape

torch.Size([500, 1, 32, 32])

In [ ]:
model.conv1.forward(features).shape

torch.Size([500, 32, 32, 32])

In [ ]:
model.conv1.kernel_size

(5, 5)

In [ ]:
conv1 = nn.Conv2d(1, 32, 5, stride=1, padding=2)
conv1.cuda()
conv1.weight = weights
conv1.forward(features).shape

torch.Size([500, 32, 32, 32])

In [ ]:
dir(model.conv1)

['__call__',
 '__class__',
 '__constants__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_name',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_modules',
 '_named_members',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_dict_hooks',
 '_tracing_name',
 '_version',
 'add_module',
 'apply',
 'bias',
 'buffers',
 'children',
 'conv2d_forward',
 'cpu',
 'cuda',
 'dilation',
 'double',
 'dump_patches',
 'eval',
 'extra_repr',
 'float',
 'forward',
 'groups',
 'ha